In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from data import gen_adult_probs

from sklearn.linear_model import LogisticRegression

In [2]:
from wasserstein_fairness.wasserstein_fairness import combined_costs, basic_costs

In [3]:
ad = gen_adult_probs(interv='pre/in')

In [4]:
ad.features = ad.features[:1000]

In [5]:
ad.protected_attributes = ad.protected_attributes[:1000]

In [6]:
ad.labels = ad.labels[:1000]

In [7]:
lrm = LogisticRegression().fit(ad.features, ad.labels)

/home/jessica/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [8]:
class JiangPenalizedLogReg():

    def __init__(self, beta, alpha, num_iter, learning_rate):
        self.beta = beta # TODO
        self.alpha = alpha # TODO
        self.lr = learning_rate
        self.num_iter = num_iter
        self.theta = None # model weights

    def fit(self, X, y, sensvec, verbose=True):
        
        # calculate barycenter step....?
        # initialize theta with output of LR
        self.theta = LogisticRegression().fit(X, y).coef_

        for i in range(self.num_iter):
            shuffleinds = np.arange(len(X))
            np.random.shuffle(shuffleinds)

            currx = X[shuffleinds]
            all_features = currx, y[shuffleinds]
            prot_feats = []
            for sens in [0,1]:
                prot_feats.append(currx[sensvec.reshape(-1) == sens])

            print(type(currx))
            print(type(self.theta))
            grad, reg_cost, wass_cost = combined_costs.gradient_line_logistic(all_features, 
                        prot_feats,theta=self.theta, beta=self.beta, alpha=self.alpha, 
                        distance='wasserstein-1')
            
            self.theta -= self.lr*grad

            if verbose and i % 1000 == 0:
                print("Round", i, ", reg cost", reg_cost, "wass cost", wass_cost)
            

    def predict_proba(self, X):
        return basic_costs.predict_prob(X, self.theta)


In [9]:
test = JiangPenalizedLogReg(beta=1, alpha=0.5, num_iter=10, learning_rate=0.01)

In [10]:
a = np.arange(5).reshape(1, -1)
a

array([[0, 1, 2, 3, 4]])

In [11]:
b = 1 + np.arange(20).reshape(5,4)
b

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20]])

In [12]:
a * b.T

array([[ 0,  5, 18, 39, 68],
       [ 0,  6, 20, 42, 72],
       [ 0,  7, 22, 45, 76],
       [ 0,  8, 24, 48, 80]])

In [13]:
b * a.reshape(-1,1)

array([[ 0,  0,  0,  0],
       [ 5,  6,  7,  8],
       [18, 20, 22, 24],
       [39, 42, 45, 48],
       [68, 72, 76, 80]])

In [14]:
# a * b.T - b * a.reshape(-1,1)

In [15]:
test.fit(ad.features, ad.labels, ad.protected_attributes)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
got data and ran model
got basic gradients
[0.32876712 0.6        1.         0.         0.43663912 0.39795918]
getting coupling
hello, log was {'cost': 0.00517647058823529}
average of coupling 0.0
sum of coupling 0
(1000, 340)
got coupling


/home/jessica/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: operands could not be broadcast together with shapes (340,1000) (1000,340) 